## Problem Set 3 (Part 2 of 2): Interacting with the Alpha Vantage API using Python libraries

In Part 2 of Problem Set 3 we will be using the data retrived from the Alpha Vantage API and creating an optimized portfolio of stocks based on Modern Portfolio Theory. The PyPortfolioOpt package will be used to optimize a portfolio of stocks based maximizing the Sharpe Ratio. This exercise is an example of how a robo-advisor would use data APIs to continuously monitor and adjusts stock portfolios. 

This part of the exercise demonstrates:

1. Setting up a programming environment to work with APIs
2. Data preprocessing to create compatible datasets
3. Using a quantititive popular portfolio optimization library to estimate risks and weights

Documentation links:
<br />Alpha Vantage: https://www.alphavantage.co/documentation/
<br />Alpha Vantage Python library: https://alpha-vantage.readthedocs.io/en/latest/#
<br />Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
<br />PyPortfolioOpt: https://pyportfolioopt.readthedocs.io/en/latest/index.html

In [1]:
api_key = "V836SIOENHQ2MSG7"

In [4]:
import pandas as pd
import requests
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

# PyPortfolioOpt libraries
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [5]:
ts = TimeSeries(key=api_key,output_format='pandas')

In [6]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [8]:
print(fb_data)

             1. open   2. high   3. low  4. close    5. volume
date                                                          
2012-05-18   42.0500   45.0000   38.000    38.232  573576400.0
2012-05-21   36.5300   36.6600   33.000    34.030  168192700.0
2012-05-22   32.6100   33.5900   30.940    31.000  101786600.0
2012-05-23   31.3700   32.5000   31.360    32.000   73600000.0
2012-05-24   32.9500   33.2100   31.770    33.030   50237200.0
2012-05-25   32.9000   32.9500   31.110    31.910   37149800.0
2012-05-29   31.4800   31.6900   28.650    28.840   78063400.0
2012-05-30   28.6937   29.5500   27.860    28.190   57267900.0
2012-05-31   28.5450   29.6700   26.830    29.600  111639200.0
2012-06-01   28.8920   29.1530   27.390    27.720   41855500.0
2012-06-04   27.2000   27.6500   26.440    26.900   35230300.0
2012-06-05   26.7000   27.7584   25.750    25.869   42473400.0
2012-06-06   26.0700   27.1700   25.520    26.810   61489200.0
2012-06-07   27.0000   27.3500   26.150    26.310   261

In [9]:
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']

Isolate "close" column of data for each security:

In [10]:
print(fb_clean)

date
2012-05-18     38.232
2012-05-21     34.030
2012-05-22     31.000
2012-05-23     32.000
2012-05-24     33.030
2012-05-25     31.910
2012-05-29     28.840
2012-05-30     28.190
2012-05-31     29.600
2012-06-01     27.720
2012-06-04     26.900
2012-06-05     25.869
2012-06-06     26.810
2012-06-07     26.310
2012-06-08     27.100
2012-06-11     27.005
2012-06-12     27.400
2012-06-13     27.270
2012-06-14     28.290
2012-06-15     30.014
2012-06-18     31.410
2012-06-19     31.911
2012-06-20     31.600
2012-06-21     31.840
2012-06-22     33.050
2012-06-25     32.060
2012-06-26     33.100
2012-06-27     32.230
2012-06-28     31.360
2012-06-29     31.095
               ...   
2019-08-14    179.710
2019-08-15    182.590
2019-08-16    183.700
2019-08-19    186.170
2019-08-20    183.810
2019-08-21    183.550
2019-08-22    182.040
2019-08-23    177.750
2019-08-26    180.360
2019-08-27    181.300
2019-08-28    181.760
2019-08-29    185.570
2019-08-30    185.670
2019-09-03    182.390
2019-

Create new dataframe with tickers as columns:

In [11]:
df = pd.DataFrame(columns=['fb','msft','aapl','amzn'])

In [12]:
print(df)

Empty DataFrame
Columns: [fb, msft, aapl, amzn]
Index: []


In [13]:
df

,fb,msft,aapl,amzn


Place data into new dataframe:

In [16]:
df['fb'] = fb_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean

In [17]:
print(df)

                 fb     msft     aapl      amzn
date                                           
2012-05-18   38.232   29.270  530.380   213.850
2012-05-21   34.030   29.750  561.280   218.110
2012-05-22   31.000   29.760  556.970   215.330
2012-05-23   32.000   29.110  570.560   217.280
2012-05-24   33.030   29.070  565.320   215.240
2012-05-25   31.910   29.060  562.290   212.890
2012-05-29   28.840   29.560  572.270   214.750
2012-05-30   28.190   29.340  579.170   209.230
2012-05-31   29.600   29.190  577.730   212.910
2012-06-01   27.720   28.450  560.990   208.220
2012-06-04   26.900   28.550  564.290   214.570
2012-06-05   25.869   28.505  562.830   213.210
2012-06-06   26.810   29.350  571.460   217.640
2012-06-07   26.310   29.230  571.720   218.800
2012-06-08   27.100   29.650  580.320   218.480
2012-06-11   27.005   28.895  571.170   216.500
2012-06-12   27.400   29.290  576.160   216.420
2012-06-13   27.270   29.130  572.160   214.730
2012-06-14   28.290   29.340  571.530   

Take only the earlist half of observations:

In [18]:
df = df[0:924]

In [19]:
df

,fb,msft,aapl,amzn
date,,,,
2012-05-18,38.232,29.270,530.380,213.850
2012-05-21,34.030,29.750,561.280,218.110
2012-05-22,31.000,29.760,556.970,215.330
2012-05-23,32.000,29.110,570.560,217.280
2012-05-24,33.030,29.070,565.320,215.240
2012-05-25,31.910,29.060,562.290,212.890
2012-05-29,28.840,29.560,572.270,214.750
2012-05-30,28.190,29.340,579.170,209.230
2012-05-31,29.600,29.190,577.730,212.910


Calculate expected returns:

In [21]:
mu = expected_returns.mean_historical_return(df)

Calculate sample covariance matrix:

In [22]:
S = risk_models.sample_cov(df)

Optimize the weights to maximzie the Sharpe Ratio:

In [26]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 28.3%
Annual volatility: 23.5%
Sharpe Ratio: 1.12


(0.2831529976215375, 0.23530861614000778, 1.1183313298862052)

In [27]:
print(weights)

{'fb': 0.23331629175290047, 'msft': 0.27282123415254717, 'aapl': 0.0, 'amzn': 0.4938624740945525}


Using optimized weights, compute portfolio accolation:

In [32]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

1 out of 4 tickers were removed
{'amzn': 859, 'msft': 5404, 'fb': 2477}


To update portfolio, call the API again for latest data and run optimization model again:

In [33]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [36]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

In [37]:
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']
##From #8


In [38]:
df['fb'] = fb_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean
print(df)

                 fb     msft     aapl      amzn
date                                           
2012-05-18   38.232   29.270  530.380   213.850
2012-05-21   34.030   29.750  561.280   218.110
2012-05-22   31.000   29.760  556.970   215.330
2012-05-23   32.000   29.110  570.560   217.280
2012-05-24   33.030   29.070  565.320   215.240
2012-05-25   31.910   29.060  562.290   212.890
2012-05-29   28.840   29.560  572.270   214.750
2012-05-30   28.190   29.340  579.170   209.230
2012-05-31   29.600   29.190  577.730   212.910
2012-06-01   27.720   28.450  560.990   208.220
2012-06-04   26.900   28.550  564.290   214.570
2012-06-05   25.869   28.505  562.830   213.210
2012-06-06   26.810   29.350  571.460   217.640
2012-06-07   26.310   29.230  571.720   218.800
2012-06-08   27.100   29.650  580.320   218.480
2012-06-11   27.005   28.895  571.170   216.500
2012-06-12   27.400   29.290  576.160   216.420
2012-06-13   27.270   29.130  572.160   214.730
2012-06-14   28.290   29.340  571.530   

In [40]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
print(weights)

Expected annual return: 28.5%
Annual volatility: 22.3%
Sharpe Ratio: 1.19
{'fb': 0.14110788492390766, 'msft': 0.42339738829620777, 'aapl': 0.0, 'amzn': 0.4354947267798846}


In [41]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

1 out of 4 tickers were removed
{'amzn': 246, 'msft': 3038, 'fb': 772}
